In [3]:
import csv
import subprocess
import cv2
import shutil
import os
from tqdm import tqdm
import multiprocessing
from multiprocessing import Pool as thread_pool

In [6]:
ROOT_DIR = 'C:/Users/utilisateur/Desktop/AS/Automatants/Yolo/'
DATA_DIR = ROOT_DIR + 'data/OPEN_IMAGES_DATASET/'
IMGS_DIR = DATA_DIR + 'imgs/'
ANNS_DIR = DATA_DIR + 'anns/'

DIRS = [IMGS_DIR, ANNS_DIR]
for DIR in DIRS:
    shutil.rmtree(DIR, ignore_errors=True)
    os.makedirs(DIR)
    
cpu_count = multiprocessing.cpu_count()
nthreads = cpu_count*2
occluded = 1 # Include occluded images
truncated = 1 # Include truncated images
groupOf = 1 # Include groupOf images
depiction = 1 # Include depiction images
inside = 1 # Include inside images
classes = ['Traffic_sign'] # Names of object classes to be downloaded

In [7]:
with open(DATA_DIR + 'class-descriptions-boxable.csv', mode='r') as infile:
    reader = csv.reader(infile)
    dict_list = {rows[1]:rows[0] for rows in reader}

for DIR in [ANNS_DIR, IMGS_DIR]:
    shutil.rmtree(DIR, ignore_errors=True)
    os.makedirs(DIR)

pool = thread_pool(nthreads)
commands = []
cnt = 0

for ind in range(0, len(classes)):
    class_name = classes[ind]
    print("Class "+str(ind) + " : " + class_name)
    
    for mode in ['train', 'validation', 'test']:
        command = "grep "+dict_list[class_name.replace('_', ' ')] + " " + DATA_DIR + "{}-annotations-bbox.csv".format(mode)
        class_annotations = subprocess.run(command.split(), stdout=subprocess.PIPE).stdout.decode('utf-8')
        class_annotations = class_annotations.splitlines()

        for line in class_annotations:

            line_parts = line.split(',')

            #IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
            if (occluded==0 and int(line_parts[8])>0):
                #print("Skipped %s",line_parts[0])
                continue
            if (truncated==0 and int(line_parts[9])>0):
                #print("Skipped %s",line_parts[0])
                continue
            if (groupOf==0 and int(line_parts[10])>0):
                #print("Skipped %s",line_parts[0])
                continue
            if (depiction==0 and int(line_parts[11])>0):
                #print("Skipped %s",line_parts[0])
                continue
            if (inside==0 and int(line_parts[12])>0):
                #print("Skipped %s",line_parts[0])
                continue

            cnt = cnt + 1
            command = 'aws s3 --no-sign-request --only-show-errors cp s3://open-images-dataset/' + mode + '/' + line_parts[0] + '.jpg ' + IMGS_DIR + line_parts[0] + '.jpg'
            commands.append(command)

            with open(ANNS_DIR + line_parts[0] + '.txt','a') as f:
                f.write(','.join(['Plate', line_parts[4], line_parts[5], line_parts[6], line_parts[7]])+'\n')

print("Annotation Count : "+str(cnt))
commands = list(set(commands))
print("Number of images to be downloaded : "+str(len(commands)))

list(tqdm(pool.imap(os.system, commands), total = len(commands) ))

pool.close()
pool.join()

print(len(os.listdir(IMGS_DIR)), 'images were downloaded')

Class 0 : Traffic_sign


FileNotFoundError: [WinError 2] Le fichier spécifié est introuvable